In [2]:
import geopandas as gpd

datapath = r"C:\Christine\Dev\camps\webapp\DGURBA-2018-01M-SH\DGURBA_2018_01M.shp"
gdfdata = gpd.read_file(datapath)
gdfdata.head()

,POPULATION,SHAPE_LEN,NUTS,NSI_CODE,SHAPE_AREA,GISCO_ID,LAT_NAT,LAU_LATIN,DGURBA,COASTAL,geometry
0,963.0,0.315585633965,AT113,10413,0.00217483024075,AT_10413,Sankt Michael im Burgenland,Sankt Michael im Burgenland,3,No,"POLYGON ((16.27358 47.14859, 16.2818 47.15034,..."
1,2658.0,0.279363929918,AT113,10414,0.00211164376227,AT_10414,Stegersbach,Stegersbach,3,No,"POLYGON ((16.14784 47.12803, 16.14241 47.13121..."
2,62.0,0.0632358779351,AT113,10423,2.04045207483E-4,AT_10423,Tschanigraben,Tschanigraben,3,No,"POLYGON ((16.32247 47.00256, 16.30328 46.99876..."
3,363.0,0.231219279656,AT111,10827,0.00110841174696,AT_10827,Weingraben,Weingraben,3,No,"POLYGON ((16.35994 47.5311, 16.38449 47.52039,..."
4,1835.0,0.660532310168,AT124,32501,0.00875004038598,AT_32501,Allentsteig,Allentsteig,3,No,"POLYGON ((15.28131 48.6446, 15.27645 48.64428,..."


In [3]:
import pandas as pd
print(gdfdata.columns)
pd.unique(gdfdata.DGURBA) #3, 2, 1
gdfdata.dtypes

Index(['POPULATION', 'SHAPE_LEN', 'NUTS', 'NSI_CODE', 'SHAPE_AREA', 'GISCO_ID',
       'LAT_NAT', 'LAU_LATIN', 'DGURBA', 'COASTAL', 'geometry'],
      dtype='object')


POPULATION      object
SHAPE_LEN       object
NUTS            object
NSI_CODE        object
SHAPE_AREA      object
GISCO_ID        object
LAT_NAT         object
LAU_LATIN       object
DGURBA           int32
COASTAL         object
geometry      geometry
dtype: object

In [16]:
gdfdata[gdfdata.DGURBA==1][['POPULATION', 'geometry']].shape

(4563, 2)

In [11]:
gdfdata[gdfdata.DGURBA==2].shape

(14491, 11)

In [ ]:
print(gdfdata.crs) #EPSG:4258
gdfdata = gdfdata.to_crs(epsg=4326)
print(gdfdata.crs)


EPSG:4258
EPSG:4326


In [20]:
gdfdata[gdfdata.DGURBA==1][['POPULATION', 'geometry']].to_file("degurba_01.geojson", driver='GeoJSON')
gdfdata[gdfdata.DGURBA==2][['POPULATION', 'geometry']].to_file("degurba_02.geojson", driver='GeoJSON')
gdfdata[gdfdata.DGURBA==3][['POPULATION', 'geometry']].to_file("degurba_03.geojson", driver='GeoJSON')

## Lire les camps et vérifier ce qui est cartographié

In [4]:
FILE_PATH = "C:\Christine\Dev\camps\webapp\camps8_18-03-2025-sansNAniOUTLIERS.csv"
camps = pd.read_csv(FILE_PATH, sep=';', encoding='utf-8', on_bad_lines='skip')

print(camps.shape)


(994, 43)


In [5]:
camps.columns

Index(['unique_id', 'nom_unique', 'camp_latitude', 'camp_longitude',
       'type_camp', 'iso3', 'prison', 'degurba', 'clc_majoritaire_13_mixte',
       'eloignementschengen', 'clc_majoritaire_2', 'clc_majoritaire_3',
       'clc_majoritaire_23_mixte', 'infrastructure_norm',
       'infrastructure_avant_conversion', 'infrastructure_solidite',
       'infrastructure_confort', 'pays_population', 'pays_surfacekm2',
       'ville_proche_population', 'distance_ville_proche', 'mairie_distance',
       'atm_distance', 'hopital_distance', 'pharmacie_distance',
       'arret_bus_distance_km', 'gare_distance_km',
       'medecin_clinique_hors_camp_distance_km',
       'dentiste_hors_camp_distance_km', 'ecole_hors_camp_distance_km',
       'poste_hors_camp_distance_km', 'distance_13_mines_decharges_chantiers',
       'distance_124_aeroport', 'distance_123_zones_portuaires',
       'distance_122_reseaux_routiers',
       'distance_24_zones_agricoles_heterogenes',
       'distance_41_zones_humides_

In [6]:
pd.unique(camps.degurba)

array(['banlieue', 'ville', 'rural', '(manquant)'], dtype=object)

In [ ]:
pd.unique(camps.prison)

array(['non', 'oui', '(manquant)', 'doute'], dtype=object)

In [8]:
pd.unique(camps.infrastructure_avant_conversion)

array(['pas de conversion', 'institut médical', 'base militaire',
       'centre de vacances', 'hôpital', 'école', 'édifice religieux',
       'gare-hôpital', 'résidence étudiante', 'hôpital psychiatrique',
       'infrastructure sportive', 'usine'], dtype=object)

In [11]:
print(pd.unique(camps.infrastructure_solidite))
print(pd.unique(camps.infrastructure_norm))


['(manquant)' '4' '2' '3' '1']
['(manquant)' 'dur / police' 'dur' 'dur / conversion' 'bateau'
 'dur / hébergement' 'mixte containers - dur / conversion'
 'mixte containers - gymnase / conversion' 'mixte containers - dur'
 'dur / enfermement' 'containers' 'containers / hangars' 'dur / transport'
 'mixte containers - tentes' 'mixte dur - rubb hall / conversion'
 'mixte dur - rubb hall' 'mixte containers - rubb hall' 'rubb hall'
 'mixte containers - dur - tentes' 'containers / transport'
 'mixte containers - dur - tentes / conversion '
 'mixte containers - bungalows' 'hangar' 'mixte dur - tentes' 'bungalows']
